In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### 학습을 위한 장치 얻기

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu" 
print("Using {} device".format(device))

Using cpu device


### 클래스 정의하기

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):    # nn.Module 상속
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [13]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)  
pred_probab = nn.Softmax(dim=1)(logits) # 각 클래스에 대한 확률로 바꿈
y_pred = pred_probab.argmax(1) # 가장 확률이 높은 인덱스 1개 뽑아냄
print('logits:{}'.format(logits))
print('pred_probab:{}'.format(pred_probab))
print("y_pred:{}".format(y_pred))

logits:tensor([[ 0.0334, -0.0630, -0.0259,  0.0530,  0.0172, -0.1379, -0.0577, -0.0708,
         -0.0124,  0.0096]], grad_fn=<AddmmBackward0>)
pred_probab:tensor([[0.1059, 0.0962, 0.0998, 0.1080, 0.1042, 0.0892, 0.0967, 0.0954, 0.1012,
         0.1034]], grad_fn=<SoftmaxBackward0>)
y_pred:tensor([3])


### 모델 계층(Layer)

In [14]:
input_image = torch.rand(3,28,28) # 예시로 28*28 크기의 이미지 3개 만들어보자
print(input_image.size())

torch.Size([3, 28, 28])


#### nn.Flatten

In [16]:
# 28,28차원을 784차원으로 변환
flatten=nn.Flatten()    
flat_image=flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#### nn.Linear

In [19]:
layer1=nn.Linear(in_features=28*28,out_features=20) # in_features=input 데이터 차원, out_features=output 데이터 차원
hidden1=layer1(flat_image) 
print(hidden1.size())

torch.Size([3, 20])


#### nn.ReLU

In [20]:
print("Before ReLU: {}\n\n".format(hidden1))
hidden1 = nn.ReLU()(hidden1)
print("After ReLU: {}".format(hidden1))  # ReLU를 적용하여 값들이 0~1사이로 바뀜

Before ReLU: tensor([[ 0.2201,  0.1618, -0.1684, -0.0122,  0.1241,  0.0067, -0.2204, -0.2363,
          0.2618,  0.2695,  0.3982, -0.3587, -0.1195,  0.6936,  0.2302,  0.0609,
         -0.6342,  0.1723,  0.0817,  0.3248],
        [ 0.3933,  0.1166, -0.1470, -0.0784,  0.4659, -0.0283, -0.0739, -0.3091,
          0.0106,  0.1180,  0.1847, -0.3550, -0.0357,  0.7508,  0.2445,  0.2790,
         -0.4287, -0.0636, -0.0862,  0.0605],
        [ 0.0721,  0.4616,  0.2859, -0.0542, -0.0344,  0.2669, -0.0046, -0.0979,
          0.2971,  0.0724, -0.0419, -0.0523,  0.0463,  0.6906,  0.3039,  0.0129,
         -0.4447,  0.0712,  0.3650, -0.0358]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2201, 0.1618, 0.0000, 0.0000, 0.1241, 0.0067, 0.0000, 0.0000, 0.2618,
         0.2695, 0.3982, 0.0000, 0.0000, 0.6936, 0.2302, 0.0609, 0.0000, 0.1723,
         0.0817, 0.3248],
        [0.3933, 0.1166, 0.0000, 0.0000, 0.4659, 0.0000, 0.0000, 0.0000, 0.0106,
         0.1180, 0.1847, 0.0000, 0.0000, 0.7508, 0.24

#### nn.Sequential

In [21]:
# 모듈 여러개를 순차적으로 쌓을 수 있음
seq_modules=nn.Sequential(flatten, layer1, nn.ReLU(),nn.Linear(20,10))
input_image=torch.rand(3,28,28)
logits=seq_modules(input_image)

#### nn.Softmax

In [22]:
# logits을 확률로 변환
softmax=nn.Softmax(dim=1)
pred_probab=softmax(logits)

### 모델 매개변수

In [23]:
print("Model structure: {}\n\n".format(model))

for name, param in model.named_parameters():
    print("Layer: {} | Size: {} | Values : {} \n".format(name,param.size(),param[:2]))

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0198, -0.0260,  0.0081,  ..., -0.0321, -0.0084,  0.0243],
        [ 0.0197, -0.0157,  0.0240,  ...,  0.0216, -0.0148, -0.0022]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0326, -0.0059], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0072,  0.0297, -0.0356,  ...,  0.0274,  0.0254, -0.0021],
        [-0.0178, -0.0103,  0.0391,  ...,  0.0219, -0.0002,  0.0391]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 